<a href="https://colab.research.google.com/github/kevin866/ml_nlp_ebaycomp/blob/spacy/ebay_ml_competition_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#import gzip
#import shutil
#with gzip.open('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv.gz', 'rb') as f_in:
 #   with open('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv', 'wb') as f_out:
 #       shutil.copyfileobj(f_in, f_out)
#unzip the gz file and copy it to the directory

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!python -m venv .env
!.env\Scripts\activate
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda116,transformers,lookups]'
!python -m spacy download en_core_web_trf

In [ ]:
import spacy
nlp = spacy.blank("en")

In [ ]:
!python -m spacy info

In [ ]:
from spacy.tokens import DocBin
db = DocBin()

In [ ]:
from tqdm import tqdm

In [ ]:
train_path = '/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Train_Tagged_Titles.tsv.gz'

In [ ]:
tags

array(['Brand', 'MPN', 'Type', 'Model', 'Color', 'Fabric Type', 'Closure',
       'Pattern', 'Material', 'No Tag', 'Product Line', 'Features',
       'Theme', 'Country/Region of Manufacture', 'Occasion', 'Department',
       'Size', 'Handle/Strap Material', 'Handle Style', 'Accents',
       'Style', 'Hardware Material', 'Trim Material',
       'Measurement, dimension', 'Pocket Type', 'Character', 'Obscure',
       'Season', 'Character Family', 'Lining Material', 'Strap Drop',
       'Handle Drop'], dtype='<U29')

In [ ]:
train_df = pd.read_csv(train_path, sep="\\\t", engine='python')


In [ ]:
train_df.head()

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,None
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model


In [ ]:
rslt_df = train_df[train_df['Tag'] == "Character"] 


In [ ]:
rslt_df.shape

(50, 4)

In [ ]:
Listing_path = '/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv'

In [ ]:
Listing_df = pd.read_csv(Listing_path, sep='\\\t', engine='python')

In [ ]:
tags = pd.DataFrame(train_df['Tag'].unique()).dropna()

In [ ]:
tags = np.ravel(np.array(tags).tolist())

In [ ]:
train = Listing_df.iloc[:5000]['Title']

In [ ]:
Listing_df.head()

,Record Number,Title
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...
1,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...
2,3,LOUIS VUITTON Damier Azur Pochette Bosphore Sh...
3,4,GUCCI Bamboo 2Way Shoulder Bag Leather Brown A...
4,5,Rank AB Vintage Gucci Sherry line PVC Leather ...


In [ ]:
train_df.head()

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,None
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model


In [ ]:
new_train = merge_tagged_identities(train_df)

In [ ]:
new_train.shape

(48200, 4)

In [ ]:
new_train.to_csv('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/train_merged_entities.tsv', sep="\t", index = False)

In [ ]:
def merge_tagged_identities(train_df):
  train_df = pd.DataFrame(train_df)
  drop = []
  processed = train_df.copy(deep=True)
  count = 0
  for i, j in train_df.iterrows():
    if j[3] == None:
      count += 1
      drop.append(i)
    elif count > 0:
      processed.iat[i-count-1,2]=" ".join([processed.iloc[i-k][2] for k in range(1,count+2)][::-1])
      count = 0
  processed=processed[processed['Tag'].notna()]
  processed = processed.reset_index(drop = True)
  return processed

In [ ]:
def extract_entites(train_df):
  processed = merge_tagged_identities(train_df)
  all_ent = []
  pos = 0
  ent = {'entites':[]}
  for i, j in processed.iterrows():
    if pos != 0:
      pos = pos + 1
    ent['entites'].append((pos, pos+len(j[2]), j[3]))
    pos = pos+len(j[2])
    if i+1 < len(processed):
      if processed.iloc[i+1]["Record Number"]>j[0]:
        pos = 0 
        all_ent.append(ent)
        ent = {'entites':[]}
  all_ent.append(ent)
  return all_ent

In [ ]:
def merge_entites_w_data(train, train_df):
  all_ent = extract_entites(train_df)
  train = pd.DataFrame(train)
  all_ent = pd.DataFrame(all_ent)
  n_train = train.join(all_ent)
  n_train['entites'] = n_train['entites'].apply(lambda x:{'entities': x})
  exam = n_train.apply(lambda x : (x[0],x[1]), axis = 1)
  exam = np.array(exam).tolist()
  return exam

In [ ]:
exam = merge_entites_w_data(train, train_df)
training = exam[:4000]
validation = exam[4000:]

In [ ]:
print(len(validation))

In [ ]:
training[0]

('LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas',
 {'entities': [(0, 13, 'Brand'),
   (14, 20, 'MPN'),
   (21, 28, 'Type'),
   (29, 38, 'Model'),
   (39, 50, 'Color'),
   (51, 57, 'Fabric Type'),
   (58, 69, 'Color'),
   (70, 76, 'Fabric Type')]})

In [ ]:
import json
def convert_to_docbin(data, tags, filename, pretrain):
  if not pretrain:
    data={"annotations":data,"classes": tags.tolist()} 
    data = json.dumps(data)
    loaded_data = json.loads(data)
    type(loaded_data) 
    for text, annot in tqdm(loaded_data["annotations"]):
      doc = nlp.make_doc(text)
      ents = []
      for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label = label, alignment_mode="contract")
        if span is None:
          print("skipping entity")
        else:
          ents.append(span)
      doc.ents = ents
      db.add(doc)
    db.to_disk("./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/{}".format(filename))
  else:
    data = {"text":data}
    for i in data:
      doc = nlp.make_doc(i)
      db.add(doc)
    db.to_disk("./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/{}".format(filename))



In [ ]:
pretrain = np.array(Listing_df.Title)


In [ ]:
nlp.make_doc(pretrain[0])

LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas

In [ ]:
convert_to_docbin(Listing_df, tags, 'pertraining.spacy', True)


In [ ]:
convert_to_docbin(training, tags, 'training.spacy', False)
convert_to_docbin(validation, tags, 'validation.spacy', False)


In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/base_config_without_trf.cfg config_pretrain.cfg --pretraining

In [ ]:
!python -m spacy pretrain config_pretrain.cfg ./output_pretrain --paths.raw_text /content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/pertraining.spacy

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_trf_v4 --paths.train ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/training.spacy --paths.dev ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/validation.spacy --gpu-id 0

✔ Created output directory:
drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_trf_v4
ℹ Saving to output directory:
drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_trf_v4
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-07 18:36:20,220] [INFO] Set up nlp object from config
[2022-08-07 18:36:20,233] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-07 18:36:20,237] [INFO] Created vocabulary
[2022-08-07 18:36:20,239] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 509kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 7.33MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 5.77MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 9.77MB/s]
Downloading: 100% 478M/478M [00:09<00:00, 51.1MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'l

In [ ]:
import spacy
nlp2 = spacy.load('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_trf_v4/model-best')

In [ ]:
#label = processed[['Record Number','Token', 'Tag']].copy()
quiz_data = Listing_df[5000:30000]

In [ ]:
def prediction(quiz_data):
  column_names = ['Record number','Aspect Name', 'Aspect Value']
  predicted = pd.DataFrame(columns = column_names)
  count = 0
  for i, j in tqdm(quiz_data.iterrows(), total=quiz_data.shape[0]):
    doc2 = nlp2(j[1])
    for ent in doc2.ents:
      predicted.loc[count] = [j[0], ent.label_, ent.text] 
      count+=1
  return predicted

In [ ]:
def drop_tag(predicted):
  predicted = predicted[predicted['Aspect Name'] != 'Obscure']
  predicted = predicted[predicted['Aspect Name'] != 'No Tag']
  predicted=predicted.set_index('Record number', drop = True)
  return predicted

In [ ]:
import spacy
nlp3 = spacy.load('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_4_1/model-best')

/usr/local/lib/python3.7/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
a = nlp3("HOBO INTERNATIONAL Women 's VINTAGE LEATHER Shoulder Bag CROSSBODY")
print(a.ents)
for ent in a.ents:
  print([ent.label_, ent.text] )


(HOBO INTERNATIONAL, Women 's, VINTAGE, LEATHER, Shoulder, Bag, CROSSBODY)
['Brand', 'HOBO INTERNATIONAL']
['Department', "Women 's"]
['No Tag', 'VINTAGE']
['Material', 'LEATHER']
['Type', 'Shoulder']
['Type', 'Bag']
['Type', 'CROSSBODY']


In [ ]:
b = nlp2("HOBO INTERNATIONAL Women 's VINTAGE LEATHER Shoulder Bag CROSSBODY")
print(b)
for ent in b.ents:
  print([ent.label_, ent.text] )


HOBO INTERNATIONAL Women 's VINTAGE LEATHER Shoulder Bag CROSSBODY
['Brand', 'HOBO']
['Accents', "'s"]
['Type', 'Bag']


In [ ]:
predicted = prediction(quiz_data)
predicted = drop_tag(predicted)

100%|██████████| 25000/25000 [37:37<00:00, 11.07it/s]


In [ ]:
predicted.head()

,Aspect Name,Aspect Value
Record number,,
5001,Brand,!
5001,Brand,Rider
5001,Model,Gaim
5002,Brand,HOBO
5002,Accents,'s


In [ ]:
predicted.to_csv('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/quiz4.tsv', header=None, sep="\t")

In [ ]:
quiz_data.head()

,Record Number,Title
5000,5001,SALE ! Kamen Rider Gaim Belt Set No. 802
5001,5002,HOBO INTERNATIONAL Women 's VINTAGE LEATHER Sh...
5002,5003,Coach Peyton Brown and Tan Signature C Shoulde...
5003,5004,MZ WALLACE Metro tote gray quilted nylon purse
5004,5005,Women Fashion Sequins Backpack Travel PU Handb...


In [ ]:
#label.columns = ["Record Number","Aspect Name","Aspect Value"]